In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/biotite-1-01-311/biotite-1.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/openmm-311/OpenMM-8.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
/kaggle/input/biotraj-311/biotraj-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/protenix/Protenix-main/CONTRIBUTING.md
/kaggle/input/protenix/Protenix-main/CODE_OF_CONDUCT.md
/kaggle/input/protenix/Protenix-main/Dockerfile
/kaggle/input/protenix/Protenix-main/.pre-commit-config.yaml
/kaggle/input/protenix/Protenix-main/inference_demo.sh
/kaggle/input/protenix/Protenix-main/LICENSE
/kaggle/input/protenix/Protenix-main/Protenix_Technical_Report.pdf
/kaggle/input/protenix/Protenix-main/.gitignore
/kaggle/input/protenix/Protenix-main/README.md
/kaggle/input/protenix/Protenix-main/requirements.txt
/kaggle/input/protenix/Protenix-main/finetune_demo.sh
/kaggle/input/protenix/Protenix-main/train_demo.sh
/kaggle/input/protenix/Protenix-main/setup.py
/kaggle/input/

In [2]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
import pickle

In [3]:
import pandas as pd
import numpy as np
import os
import sys
import argparse

In [4]:
!pip install /kaggle/input/openmm-311/OpenMM-8.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl

Processing /kaggle/input/openmm-311/OpenMM-8.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl


In [5]:
!pip install /kaggle/input/simtk-0-1/simtk-0.1.0-py2.py3-none-any.whl

Processing /kaggle/input/simtk-0-1/simtk-0.1.0-py2.py3-none-any.whl


In [6]:
!pip install /kaggle/input/pytest-runner/pytest_runner-6.0.1-py3-none-any.whl

Processing /kaggle/input/pytest-runner/pytest_runner-6.0.1-py3-none-any.whl


In [7]:
!pip install /kaggle/input/biopython-311/biopython-1.85-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/biopython-311/biopython-1.85-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [8]:
!pip install /kaggle/input/ml-collections/ml_collections-1.0.0-py3-none-any.whl

Processing /kaggle/input/ml-collections/ml_collections-1.0.0-py3-none-any.whl


In [9]:
###drfold2###

In [10]:
import scipy

In [11]:
from datetime import datetime
import pytz
import Bio

In [12]:
import os,sys

import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.expand_frame_repr', False)

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from timeit import default_timer as timer
import re

import matplotlib 
import matplotlib.pyplot as plt


# helper--
class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__

	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

def time_to_str(t, mode='min'):
	if mode=='min':
		t  = int(t)/60
		hr = t//60
		min = t%60
		return '%2d hr %02d min'%(hr,min) 
	elif mode=='sec':
		t   = int(t)
		min = t//60
		sec = t%60
		return '%2d min %02d sec'%(min,sec)

	else:
		raise NotImplementedError

def gpu_memory_use():
    if torch.cuda.is_available():
        device = torch.device(0)
        free, total = torch.cuda.mem_get_info(device)
        used= (total - free) / 1024 ** 3
        return int(round(used))
    else:
        return 0

def set_aspect_equal(ax):
	x_limits = ax.get_xlim()
	y_limits = ax.get_ylim()
	z_limits = ax.get_zlim()

	# Compute the mean of each axis
	x_middle = np.mean(x_limits)
	y_middle = np.mean(y_limits)
	z_middle = np.mean(z_limits)

	# Compute the max range across all axes
	max_range = max(x_limits[1] - x_limits[0],
					y_limits[1] - y_limits[0],
					z_limits[1] - z_limits[0]) / 2.0

	# Set the new limits to ensure equal scaling
	ax.set_xlim(x_middle - max_range, x_middle + max_range)
	ax.set_ylim(y_middle - max_range, y_middle + max_range)
	ax.set_zlim(z_middle - max_range, z_middle + max_range)


print('torch',torch.__version__)
print('torch.cuda',torch.version.cuda)

print('IMPORT OK!!!')

torch 2.6.0+cu124
torch.cuda 12.4
IMPORT OK!!!


In [13]:
MODE = 'submit' #'local' # submit


DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'
if MODE == 'local':
    valid_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/validation_sequences.csv')
    label_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/validation_labels.csv')
    label_df['target_id'] = label_df['ID'].apply(lambda x: '_'.join(x.split('_')[:-1]))

if MODE == 'submit':
    valid_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/test_sequences.csv')

print('len(valid_df)',len(valid_df))
print(valid_df.iloc[0])
print('')




DEVICE='cuda' #'cpu'

print('MODE:', MODE)
print('SETTING OK!!!')

len(valid_df) 12
target_id                                                      R1107
sequence           GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUU...
temporal_cutoff                                           2022-05-28
description        CPEB3 ribozyme\nHuman\nhuman CPEB3 HDV-like ri...
all_sequences      >7QR4_1|Chain A|U1 small nuclear ribonucleopro...
Name: 0, dtype: object

MODE: submit
SETTING OK!!!


In [14]:
from pathlib import Path
from Bio.PDB import PDBParser

In [15]:
def Get_other(seq, basenpy_standard):
    """
    OTHER_COOR用の座標を取得する関数
    :param seq: 塩基配列（例: 'AGCT'）
    :param basenpy_standard: 塩基ごとの座標 (5, 3) の配列
    :return: 塩基配列に対応した座標を持つ配列 (L, 5, 3)
    """
    basenpy = np.zeros([len(seq), 5, 3])  # L, 5, 3 の配列を用意
    seqnpy = np.array(list(seq))  # 塩基配列をリストに変換

    # 各塩基 ('A', 'T', 'C', 'G') に対応する座標を設定
    basenpy[seqnpy == 'A'] = basenpy_standard[0]  # 'A'に対応する座標
    basenpy[seqnpy == 'T'] = basenpy_standard[1]  # 'T'に対応する座標
    basenpy[seqnpy == 'C'] = basenpy_standard[2]  # 'C'に対応する座標
    basenpy[seqnpy == 'G'] = basenpy_standard[3]  # 'G'に対応する座標

    return basenpy

In [16]:
def write_frame_coor_to_pdb(coor, seq, pdbfile):

    # coor = out['coor']
    #tempray solution
    #todo: frame(P,C4,N) to C1
    tx = coor.astype(np.float64)
    basex = Get_base(seq, BASE_COOR)  # L,3,3
    otherx = Get_other(seq, OTHER_COOR)  # L,5,3

    tx = torch.from_numpy(tx)
    basex = torch.from_numpy(basex)
    otherx = torch.from_numpy(otherx)

    #init_quat
    L= len(seq)
    x = torch.rand([L, 21])
    x[:, 18:] = tx.mean(dim=1)

    biasq = torch.mean(tx, dim=1, keepdim=True)
    q = tx - biasq
    m = torch.einsum('bnz,bny->bzy', basex, q).reshape([L, -1])
    x[:, :9] = x[:, 9:18] = m
    rama = x.double()

    # rama=torch.cat([rama.view(self.L,2),self.betas],dim=-1)
    xyz = quat2b(basex, rama.view(L, 21)[:, 9:]).float().data.numpy()
    other_xyz = quat2b(otherx, rama.view(L, 21)[:, 9:]).float().data.numpy()

    atom_name = [' P  ', " C4'", ' N1/N9 ']
    last_name = ['P', 'C', 'N']
    other_atom_name = [" O5'", " C5'", " C3'", " O3'", " C1'"]
    other_last_name = ['O', "C", "C", "O", "C"]

    lines = [f'REMARK (todo: print energy value))']
    templet = '%6s%5d %4s %3s %1s%4d    %8.3f%8.3f%8.3f%6.2f%6.2f          %2s%2s'

    L=len(seq)
    count = 1
    for i in range(L):
        if seq[i] in ['a', 'g', 'A', 'G']:
            atom_name = [' P  ', " C4'", ' N9 ']
        elif seq[i] in ['c', 'u', 'C', 'U']:
            atom_name = [' P  ', " C4'", ' N1 ']

        for j in range(coor.shape[1]):
            outs = ('ATOM  ', count, atom_name[j], seq[i], 'A', i + 1, xyz[i][j][0], xyz[i][j][1],
                    xyz[i][j][2], 0, 0, last_name[j], '')
            lines.append(templet % outs)
            count += 1

        for j in range(other_xyz.shape[1]):
            outs = (
                'ATOM  ', count, other_atom_name[j], seq[i], 'A', i + 1, other_xyz[i][j][0],
                other_xyz[i][j][1],
                other_xyz[i][j][2], 0, 0, other_last_name[j], '')
            lines.append(templet % outs)
            count += 1

    lines = '\n'.join(lines)
    file = open(pdbfile, 'w')
    file.write(lines)
    file.close()

In [17]:
import numpy as np

# ファイルのパスを指定
file_path = '/kaggle/input/drfold2/PotentialFold/lib/base.npy'

# base.npyを読み込む
base_data = np.load(file_path)

# 読み込んだデータを確認
print(base_data)

[[[ 2.3240912  -0.80317456 -1.5209152 ]
  [-0.8031747   1.3708649  -0.56768906]
  [-1.520915   -0.5676891   2.088603  ]]

 [[ 2.3250363  -0.80280226 -1.5222337 ]
  [-0.8028024   1.3702725  -0.5674678 ]
  [-1.5222336  -0.5674678   2.0897026 ]]

 [[ 2.35021    -0.7880346  -1.5621773 ]
  [-0.78803474  1.3541472  -0.5661155 ]
  [-1.5621774  -0.56611556  2.1282897 ]]

 [[ 2.3493478  -0.7881313  -1.5612125 ]
  [-0.7881318   1.3542684  -0.5661326 ]
  [-1.5612122  -0.5661328   2.1273444 ]]]


In [18]:
BASE_COOR = np.array([[[ 2.3240912, -0.80317456, -1.5209152 ],
                       [-0.8031747,  1.3708649, -0.56768906],
                       [-1.520915,  -0.5676891,  2.088603 ]],

                      [[ 2.3250363, -0.80280226, -1.5222337 ],
                       [-0.8028024,  1.3702725, -0.5674678 ],
                       [-1.5222336, -0.5674678,  2.0897026 ]],

                      [[ 2.35021,   -0.7880346,  -1.5621773 ],
                       [-0.78803474, 1.3541472, -0.5661155 ],
                       [-1.5621774, -0.56611556,  2.1282897 ]],

                      [[ 2.3493478, -0.7881313, -1.5612125 ],
                       [-0.7881318,  1.3542684, -0.5661326 ],
                       [-1.5612122, -0.5661328,  2.1273444 ]]])

In [19]:
import numpy as np

# ファイルのパスを指定
file_path = '/kaggle/input/drfold2/PotentialFold/lib/other2.npy'

# base.npyを読み込む
base_data = np.load(file_path)

# 読み込んだデータを確認
print(base_data)

[[[ 1.1992943   0.02163811 -0.74968845]
  [ 0.31570393  0.88175213 -1.4532801 ]
  [-0.38125393  2.0709064   0.71517557]
  [-0.03065619  3.4296124   0.5162348 ]
  [-2.0603676   0.4805917   1.2017885 ]]

 [[ 1.201576    0.0235323  -0.75093365]
  [ 0.31460416  0.8799998  -1.454717  ]
  [-0.378845    2.0691633   0.7153938 ]
  [-0.0269871   3.4278457   0.5148757 ]
  [-2.0596368   0.48132515  1.2025663 ]]

 [[ 1.2404578   0.0441614  -0.7788638 ]
  [ 0.30463168  0.85085434 -1.4778417 ]
  [-0.33137608  2.0489094   0.7070306 ]
  [ 0.04332279  3.3994405   0.49545667]
  [-2.046944    0.50140464  1.2187861 ]]

 [[ 1.2388773   0.0426251  -0.7780464 ]
  [ 0.30505154  0.85181475 -1.4766837 ]
  [-0.33256125  2.0494554   0.70743614]
  [ 0.04215779  3.3991168   0.4981761 ]
  [-2.0471878   0.5006145   1.218904  ]]]


In [20]:
OTHER_COOR = np.array([[[ 1.1992943,   0.02163811, -0.74968845],
                        [ 0.31570393,  0.88175213, -1.4532801 ],
                        [-0.38125393,  2.0709064,   0.71517557],
                        [-0.03065619,  3.4296124,   0.5162348 ],
                        [-2.0603676,   0.4805917,   1.2017885 ]],

                       [[ 1.201576,    0.0235323,  -0.75093365],
                        [ 0.31460416,  0.8799998,  -1.454717  ],
                        [-0.378845,    2.0691633,   0.7153938 ],
                        [-0.0269871,   3.4278457,   0.5148757 ],
                        [-2.0596368,   0.48132515,  1.2025663 ]],

                       [[ 1.2404578,   0.0441614,  -0.7788638 ],
                        [ 0.30463168,  0.85085434, -1.4778417 ],
                        [-0.33137608,  2.0489094,   0.7070306 ],
                        [ 0.04332279,  3.3994405,   0.49545667],
                        [-2.046944,    0.50140464,  1.2187861 ]],

                       [[ 1.2388773,   0.0426251,  -0.7780464 ],
                        [ 0.30505154,  0.85181475, -1.4766837 ],
                        [-0.33256125,  2.0494554,   0.70743614],
                        [ 0.04215779,  3.3991168,   0.4981761 ],
                        [-2.0471878,   0.5006145,   1.218904  ]]])

In [21]:
def parse_pdb_to_xyz(pdb_file):
    """
    PDBファイルを解析し、各原子の座標（xyz）、残基名（resname）、残基番号（resid）を返します。
    
    :param pdb_file: 解析するPDBファイルのパス
    :return: 座標（xyz）、残基名（resname）、残基番号（resid）のタプル
    """
    # 初期化
    xyz = []
    resname = []
    resid = []
    
    # PDBファイルを開いて解析
    with open(pdb_file, 'r') as f:
        for line in f:
            # ATOMまたはHETATM行のみを対象にする
            if line.startswith('ATOM') or line.startswith('HETATM'):
                # 座標（x, y, z）はPDBファイルの 30-38, 38-46, 46-54 文字位置にある
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                
                # 残基名（resname）はPDBファイルの 17-20 文字位置にある
                res = line[17:20].strip()

                # 残基番号（resid）はPDBファイルの 22-26 文字位置にある
                res_id = int(line[22:26].strip())
                
                # 座標と残基情報をリストに追加
                xyz.append([x, y, z])
                resname.append(res)
                resid.append(res_id)
    
    # numpy配列に変換して返す
    return np.array(xyz), resname, resid

In [22]:
def solution_to_submit_df(solution):
    submit_df = []
    for k,s in solution.items():
        df = coord_to_df(s['sequence'], s['coord'], s['target_id'])
        submit_df.append(df)
    
    submit_df = pd.concat(submit_df)
    return submit_df
 

def coord_to_df(sequence, coord, target_id):
    L = len(sequence)
    df = pd.DataFrame()
    df['ID'] = [f'{target_id}_{i + 1}' for i in range(L)]
    df['resname'] = [s for s in sequence]
    df['resid'] = [i + 1 for i in range(L)]

    num_coord = len(coord)
    for j in range(num_coord):
        df[f'x_{j+1}'] = coord[j][:, 0]
        df[f'y_{j+1}'] = coord[j][:, 1]
        df[f'z_{j+1}'] = coord[j][:, 2]
    return df

In [23]:
def quat2b(baseframe,x):
    #x: L 6
    L = x.shape[0]
    #predx=( baseframe.repeat(L,1,1) )
    #rot = quat2rot(x[:,:3],L)
    rot = Non2rot(x[:,:9],L)
    
    trans=x[:,9:]
    predx = batch_atom_transform(baseframe,rot,trans)
    return predx

In [24]:
def Non2rot(x,L):
    """Maps 9D input vectors onto SO(3) via symmetric orthogonalization.

    x: should have size [batch_size, 9]

    Output has size [batch_size, 3, 3], where each inner 3x3 matrix is in SO(3).
    """
    m = x.view(-1, 3, 3)
    u, s, v = torch.svd(m)
    vt = torch.transpose(v, 1, 2)
    det = torch.det(torch.matmul(u, vt))
    det = det.view(-1, 1, 1)
    vt = torch.cat((vt[:, :2, :], vt[:, -1:, :] * det), 1)
    r = torch.matmul(u, vt)
    return r

In [25]:
def batch_atom_transform(k,rotation,translation):
    # k:            L N 3
    # rotation:     L 3 x 3
    # translation:  L 3
    return torch.einsum('bja,bad->bjd',k,rotation) + translation[:,None,:]

In [26]:
def frame_coor_to_C1(coor, sequence):
    """
    coor: np.ndarray or torch.Tensor, shape (L, 3, 3)
        各残基の予測フレーム座標（例：P, C4', N の3点）
    sequence: str
        RNA配列（例えば "ACGUU" など）
    
    戻り値: np.ndarray, shape (L, 3)
        各残基の C1' 原子の3次元座標
    """
    tx = coor.astype(np.float64)
    basex = Get_base(sequence, BASE_COOR)  # (L, 3, 3)
    otherx = Get_other(sequence, OTHER_COOR)  # (L, 5, 3)

    tx = torch.from_numpy(tx)
    basex = torch.from_numpy(basex)
    otherx = torch.from_numpy(otherx)

    L = len(sequence)
    x = torch.rand([L, 21])
    x[:, 18:] = tx.mean(dim=1)

    biasq = torch.mean(tx, dim=1, keepdim=True)
    q = tx - biasq
    m = torch.einsum('bnz,bny->bzy', basex, q).reshape([L, -1])
    x[:, :9] = x[:, 9:18] = m
    rama = x.double()

    other_xyz = quat2b(otherx, rama.view(L, 21)[:, 9:]).float().data.numpy()
    c1_xyz = other_xyz[:, 4, :]  # ← C1'はインデックス4

    return c1_xyz  # shape (L, 3)

In [27]:
def vdw_penalty(coor, epsilon=0.1, sigma=2.0):
    """
    簡略化したLennard-Jones的 VDW ペナルティ
    """
    C4 = coor[:, 0, :]
    diff = C4[:, None, :] - C4[None, :, :]
    dists = np.linalg.norm(diff, axis=-1) + 1e-6
    mask = (np.abs(np.arange(C4.shape[0])[:, None] - np.arange(C4.shape[0])[None, :]) > 1)
    dists = dists[mask]
    lj = 4 * epsilon * ((sigma / dists)**12 - (sigma / dists)**6)
    lj[dists > 6.0] = 0  # 長距離は打ち切り
    return np.sum(lj)
#実験的に sigma=2.5 や epsilon=0.2 にしてみるのも良い
    
def hydrogen_bond_score(coor):
    C4 = coor[:, 0, :]
    L = C4.shape[0]
    penalty = 0.0
    for i in range(L):
        for j in range(L):
            if abs(i - j) <= 1:
                continue
            dist = np.linalg.norm(C4[i] - C4[j])
            if 2.6 < dist < 3.2:
                # 方向性がないため補正が必要
                penalty += (dist - 2.9)**2
    return penalty

# 各塩基に対する簡易な部分電荷（例示）
BASE_CHARGES = {'A': +0.1, 'C': -0.1, 'G': +0.2, 'U': -0.2}

def electrostatic_penalty(coor, sequence, k=1.0):
    C4 = coor[:, 0, :]
    charges = np.array([BASE_CHARGES.get(b, 0.0) for b in sequence])
    diff = C4[:, None, :] - C4[None, :, :]
    dists = np.linalg.norm(diff, axis=-1) + 1e-6
    qiqj = charges[:, None] * charges[None, :]
    mask = (np.abs(np.arange(len(sequence))[:, None] - np.arange(len(sequence))[None, :]) > 1)
    penalty = np.sum((qiqj / dists)[mask])
    return penalty

def hydrogen_bond_directionality_score(coor):
    """
    簡易な水素結合の方向性スコア。
    C4' → C1' → C4' の角度を180度に近いと良いと仮定（ラジアンでπ）
    """
    C4 = coor[:, 0, :]  # 仮に C4' をドナー/アクセプター位置
    C1 = coor[:, 2, :]  # C1' を仮想の水素方向とみなす

    L = C4.shape[0]
    total_penalty = 0.0
    count = 0

    for i in range(L):
        for j in range(L):
            if i == j or abs(i - j) <= 1:
                continue
            vec_donor = C4[i]
            vec_h = C1[i]  # 仮想H方向
            vec_acceptor = C4[j]
            dist = np.linalg.norm(vec_donor - vec_acceptor)
            if 2.8 <= dist <= 3.2:
                angle = bond_angle(vec_donor, vec_h, vec_acceptor)  # Hを頂点とした角
                penalty = (angle - np.pi) ** 2  # π = 180度
                total_penalty += penalty
                count += 1

    if count == 0:
        return 0.0
    return total_penalty / count  # 平均ペナルティ

import numpy as np

def bond_angle(a, b, c):
    # ∠abc の角度（ラジアン）
    ba = a - b
    bc = c - b
    cosine_angle = np.sum(ba * bc, axis=-1) / (
        np.linalg.norm(ba, axis=-1) * np.linalg.norm(bc, axis=-1) + 1e-8
    )
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.arccos(cosine_angle)
    return angle  # ラジアン

def torsion_angle(a, b, c, d):
    # ねじれ角 a-b-c-d（ラジアン）
    b1 = b - a
    b2 = c - b
    b3 = d - c
    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)
    n1 /= np.linalg.norm(n1, axis=-1, keepdims=True) + 1e-8
    n2 /= np.linalg.norm(n2, axis=-1, keepdims=True) + 1e-8
    m1 = np.cross(n1, b2 / (np.linalg.norm(b2, axis=-1, keepdims=True) + 1e-8))
    x = np.sum(n1 * n2, axis=-1)
    y = np.sum(m1 * n2, axis=-1)
    return np.arctan2(y, x)

def score_energy_one(seq, target_id, out):
    coor = out['coor']  # shape: (L, 3, 3)
    if coor.ndim != 3 or coor.shape[1:] != (3, 3):
        print(f"[WARNING] Unexpected shape for coor: {coor.shape}")
        return 1e6

    C4 = coor[:, 0, :]  # shape: (L, 3)
    L = C4.shape[0]

    # 距離
    diff = C4[:, None, :] - C4[None, :, :]
    dists = np.linalg.norm(diff, axis=-1)
    mask = (dists < 3.0) & (dists > 0.1) & (np.abs(np.arange(L)[:, None] - np.arange(L)[None, :]) > 1)
    dist_penalty = np.sum(mask)

    # 内角
    angle_penalty = sum(
        (bond_angle(C4[i-1], C4[i], C4[i+1]) - np.radians(105)) ** 2
        for i in range(1, L-1)
    )

    # ねじれ
    torsion_penalty = sum(
        torsion_angle(C4[i-1], C4[i], C4[i+1], C4[i+2]) ** 2
        for i in range(1, L-2)
    )

    # 追加項目
    vdw = vdw_penalty(coor)
    hbond = hydrogen_bond_score(coor)
    electro = electrostatic_penalty(coor, seq)
    hbond_dir = hydrogen_bond_directionality_score(coor)


    # 各項の重み付け（調整可能）
    total_penalty = (
        dist_penalty +
        angle_penalty +
        torsion_penalty +
        0.5 * vdw +
        0.2 * hbond +
        0.05 * electro + 
        0.2 * hbond_dir

    )
    return float(total_penalty)

In [28]:
sys.path.append('/kaggle/input/drfold2/cfg_97')
sys.path.append('/kaggle/input/drfold2/PotentialFold')
from EvoMSA2XYZ import MSA2XYZ
from RNALM2.Model import RNA2nd
from data import parse_seq, Get_base






###########################################################3



   



# data helper
def make_data(seq):
    aa_type = parse_seq(seq)
    base = Get_base(seq, BASE_COOR)
    seq_idx = np.arange(len(seq)) + 1

    msa = aa_type[None, :]
    msa = torch.from_numpy(msa)
    msa = torch.cat([msa, msa], 0) #???
    msa = F.one_hot(msa.long(), 6).float()

    base_x  = torch.from_numpy(base).float()
    seq_idx = torch.from_numpy(seq_idx).long()
    return msa, base_x, seq_idx
    
def make_dummy_solution():
    solution=dotdict()
    for i, row in valid_df.iterrows():
        target_id = row.target_id
        sequence = row.sequence
        solution[target_id]=dotdict(
            target_id=target_id,
            sequence=sequence,
            coord=[],
        )
    return solution

out_dir = '/kaggle/working/model-output'
os.makedirs(out_dir, exist_ok=True)
solution = make_dummy_solution()

will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint


In [29]:
###protenix###

In [30]:
MODEL_TYPE='protenix'

In [31]:
! mkdir /af3-dev 
! ln -s /kaggle/input/protenix-checkpoints /af3-dev/release_data
! ls /af3-dev/release_data/

components.v20240608.cif		model_v0.2.0.pt
components.v20240608.cif.rdkit_mol.pkl


In [32]:
import Bio

from copy import deepcopy

import pandas as pd
from Bio.PDB import Atom, Model, Chain, Residue, Structure, PDBParser
from Bio import SeqIO
import os, sys
import re
import numpy as np
import torch

import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
time0=time.time()

print('IMPORT OK !!!!')

IMPORT OK !!!!


In [33]:
PYTHON = sys.executable
print('PYTHON',PYTHON)


DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'
# helper ----
class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__

	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

# visualisation helper ----
def set_aspect_equal(ax):
	x_limits = ax.get_xlim()
	y_limits = ax.get_ylim()
	z_limits = ax.get_zlim()

	# Compute the mean of each axis
	x_middle = np.mean(x_limits)
	y_middle = np.mean(y_limits)
	z_middle = np.mean(z_limits)

	# Compute the max range across all axes
	max_range = max(x_limits[1] - x_limits[0],
					y_limits[1] - y_limits[0],
					z_limits[1] - z_limits[0]) / 2.0

	# Set the new limits to ensure equal scaling
	ax.set_xlim(x_middle - max_range, x_middle + max_range)
	ax.set_ylim(y_middle - max_range, y_middle + max_range)
	ax.set_zlim(z_middle - max_range, z_middle + max_range)




# xyz df helper --------------------
def get_truth_df(target_id):
    truth_df = LABEL_DF[LABEL_DF['target_id'] == target_id]
    truth_df = truth_df.reset_index(drop=True)
    return truth_df

def parse_output_to_df(output, seq, target_id):
    df = []
    chain_data = []
    for i, res in enumerate(seq):
        d=dict(ID = target_id,
                    resname=res,
                    resid=i+1)
        for n in range(len(output)):
            d={**d, f'x_{n+1}': round(output[n,i,0].item(),3),
                     f'y_{n+1}': round(output[n,i,1].item(),3),
                     f'z_{n+1}': round(output[n,i,2].item(),3)}
        chain_data.append(d)

    if len(chain_data)!=0:
        chain_df = pd.DataFrame(chain_data)
        df.append(chain_df)
        ##print(chain_df)
    return df

def parse_pdb_to_df(pdb_file, target_id):
    parser = PDBParser()
    structure = parser.get_structure('', pdb_file)

    df = []
    for model in structure:
        for chain in model:
            print(chain)
            chain_data = []
            for residue in chain:
                # print(residue)
                if residue.get_resname() in ['A', 'U', 'G', 'C']:
                    # Check if the residue has a C1' atom
                    if 'C1\'' in residue:
                        atom = residue['C1\'']
                        xyz = atom.get_coord()
                        resname = residue.get_resname()
                        resid = residue.get_id()[1]

                        #todo detect discontinous: resid = prev_resid+1
                        #ID	resname	resid	x_1	y_1	z_1
                        chain_data.append(dict(
                            ID = target_id+'_'+str(resid),
                            resname=resname,
                            resid=resid,
                            x_1=xyz[0],
                            y_1=xyz[1],
                            z_1=xyz[2],
                        ))
                        ##print(f"Residue {resname} {resid}, Atom: {atom.get_name()}, xyz: {xyz}")

            if len(chain_data)!=0:
                chain_df = pd.DataFrame(chain_data)
                df.append(chain_df)
                ##print(chain_df)
    return df


# usalign helper --------------------
def write_target_line(
    atom_name, atom_serial, residue_name, chain_id, residue_num, x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='P'
):
    """
    Writes a single line of PDB format based on provided atom information.

    Args:
        atom_name (str): Name of the atom (e.g., "N", "CA").
        atom_serial (int): Atom serial number.
        residue_name (str): Residue name (e.g., "ALA").
        chain_id (str): Chain identifier.
        residue_num (int): Residue number.
        x_coord (float): X coordinate.
        y_coord (float): Y coordinate.
        z_coord (float): Z coordinate.
        occupancy (float, optional): Occupancy value (default: 1.0).
        b_factor (float, optional): B-factor value (default: 0.0).

    Returns:
        str: A single line of PDB string.
    """
    return f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} {residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}{z_coord:>8.3f}{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'

def write_xyz_to_pdb(df, pdb_file, xyz_id = 1):
    resolved_cnt = 0
    with open(pdb_file, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row[f'x_{xyz_id}']
            y_coord = row[f'y_{xyz_id}']
            z_coord = row[f'z_{xyz_id}']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt

def parse_usalign_for_tm_score(output):
    # Extract TM-score based on length of reference structure (second)
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)[1]
    if not tm_score_match:
        raise ValueError('No TM score found')
    return float(tm_score_match)

def parse_usalign_for_transform(output):
    # Locate the rotation matrix section
    matrix_lines = []
    found_matrix = False

    for line in output.splitlines():
        if "The rotation matrix to rotate Structure_1 to Structure_2" in line:
            found_matrix = True
        elif found_matrix and re.match(r'^\d+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+$', line):
            matrix_lines.append(line)
        elif found_matrix and not line.strip():
            break  # Stop parsing if an empty line is encountered after the matrix

    # Parse the rotation matrix values
    rotation_matrix = []
    for line in matrix_lines:
        parts = line.split()
        row_values = list(map(float, parts[1:]))  # Skip the first column (index)
        rotation_matrix.append(row_values)

    return np.array(rotation_matrix)

def call_usalign(predict_df, truth_df, verbose=1):
    truth_pdb = '~truth.pdb'
    predict_pdb = '~predict.pdb'
    write_xyz_to_pdb(predict_df, predict_pdb, xyz_id=1)
    write_xyz_to_pdb(truth_df, truth_pdb, xyz_id=1)

    command = f'{USALIGN} {predict_pdb} {truth_pdb} -atom " C1\'" -m -'
    output = os.popen(command).read()
    if verbose==1:
        print(output)
    tm_score = parse_usalign_for_tm_score(output)
    transform = parse_usalign_for_transform(output)
    return tm_score, transform

print('HELPER OK!!!')

PYTHON /usr/bin/python3
HELPER OK!!!


In [34]:
import os,sys
sys.path.append('/kaggle/input/protenix/Protenix-main/runner')
sys.path.append('/kaggle/input/protenix/Protenix-main')

In [35]:
!pip install /kaggle/input/remit-311/rdkit-2024.9.6-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/remit-311/rdkit-2024.9.6-cp311-cp311-manylinux_2_28_x86_64.whl


In [36]:
!pip install /kaggle/input/biotraj-311/biotraj-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/biotraj-311/biotraj-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [37]:
!pip install /kaggle/input/biotite-1-01-311/biotite-1.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/biotite-1-01-311/biotite-1.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [38]:
if MODEL_TYPE=='protenix':
    
    
    from runner.batch_inference import get_default_runner
    from runner.inference import update_inference_configs, InferenceRunner

    from protenix.data.infer_data_pipeline import InferenceDataset

    np.random.seed(0)
    torch.random.manual_seed(0)
    torch.cuda.manual_seed_all(0)

    class DictDataset(InferenceDataset):
        def __init__(
            self,
            seq_list: list,
            dump_dir: str,
            id_list: list = None,
            use_msa: bool = False,
        ) -> None:

            self.dump_dir = dump_dir
            self.use_msa = use_msa
            if isinstance(id_list,type(None)):
                self.inputs = [{"sequences": 
                                [{"rnaSequence": 
                                  {"sequence": seq, 
                                   "count": 1}}],
                                "name": "query"} for seq in seq_list]
            else:
                self.inputs = [{"sequences": 
                                [{"rnaSequence": 
                                  {"sequence": seq, 
                                   "count": 1}}],
                                "name": i} for i, seq in zip(id_list,seq_list)]

In [39]:
# モデル初期化（drfold2用）
def init_drfold2_models():
    rnalm = RNA2nd(dict(
        s_in_dim=5,
        z_in_dim=2,
        s_dim=512,
        z_dim=128,
        N_elayers=18,
    ))
    rnalm.load_state_dict(torch.load(
        '/kaggle/input/drfold2-models/model_hub/RCLM/epoch_67000',
        map_location='cpu',
        weights_only=True), strict=False)
    rnalm = rnalm.to(DEVICE).eval()

    msa2xyz = MSA2XYZ(
        seq_dim=6,
        msa_dim=7,
        N_ensemble=1,
        N_cycle=8,
        m_dim=64,
        s_dim=64,
        z_dim=64,
    )
    msa2xyz_file = [
        f'/kaggle/input/drfold2-models/model_hub/cfg_97/model_{i}' for i in range(20)
    ]
    msa2xyz_state_dict = [torch.load(f, map_location='cpu', weights_only=True) for f in msa2xyz_file]
    msa2xyz.load_state_dict(msa2xyz_state_dict[0], strict=True)
    msa2xyz = msa2xyz.to(DEVICE).eval()
    msa2xyz.msaxyzone.premsa.rnalm = rnalm

    return msa2xyz, msa2xyz_state_dict

# モデル初期化（protenix用）
def init_protenix_runner():
    from configs.configs_base import configs as configs_base
    from configs.configs_data import data_configs
    from configs.configs_inference import inference_configs
    from protenix.config.config import parse_configs
    

    configs_base["use_deepspeed_evo_attention"] = (
        os.environ.get("USE_DEEPSPEED_EVO_ATTENTION", False) == "true")
    configs_base["model"]["N_cycle"] = 10
    configs_base["sample_diffusion"]["N_sample"] = 5
    configs_base["sample_diffusion"]["N_step"] = 200
    inference_configs['load_checkpoint_path'] = '/kaggle/input/protenix-checkpoints/model_v0.2.0.pt'
    configs = {**configs_base, **{"data": data_configs}, **inference_configs}

    configs = parse_configs(
        configs=configs,
        fill_required_with_null=True,
    )

    return InferenceRunner(configs), configs

In [40]:
###先頭MAX_LENGTH分を抽出###
MAX_LENGTH = 900  

valid_df1 = valid_df.copy()
valid_df1['sequence'] = valid_df1['sequence'].str[:MAX_LENGTH]
valid_df1 = valid_df1.reset_index(drop=True)

In [41]:
import time
from timeit import default_timer as timer
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd

def run_prediction(valid_df1):
    start_total = timer()
    msa2xyz, msa2xyz_state_dict = init_drfold2_models()
    runner, configs = init_protenix_runner()

    for i, row in tqdm(valid_df1.iterrows(), total=len(valid_df), desc="Predicting"):
        target_id = row.target_id
        sequence = row.sequence
        seq_len = len(sequence)
        L = len(sequence)

        all_coordinates = []

        if seq_len <= 480:
            # === drfold2で3構造生成 ===
            start_drfold = timer()

            msa, base_x, seq_idx = make_data(sequence)
            device = next(msa2xyz.parameters()).device
            msa, base_x, seq_idx = msa.to(device), base_x.to(device), seq_idx.to(device)

            coordinate = []
            energy = []
            model_to_try = list(range(20))

            for c in model_to_try:
                msa2xyz.load_state_dict(msa2xyz_state_dict[c], strict=False)

                with torch.no_grad():
                    out = msa2xyz.pred(msa, seq_idx, None, base_x, np.array(list(sequence)))

                if L != len(sequence):
                    out['coor'] = np.pad(out['coor'], ((0, L - len(sequence)), (0, 0), (0, 0)), 'constant')


                e = score_energy_one(sequence, target_id, out)
                energy.append(e)

                xyz = frame_coor_to_C1(out['coor'], sequence)
                coordinate.append(xyz)

            top3_idx = np.argsort(energy)[:3]
            drfold_coords = [coordinate[k] for k in top3_idx]
            all_coordinates.extend(drfold_coords)

            torch.cuda.empty_cache()
            end_drfold = timer()
            print(f"[{target_id}] DRfold2 prediction time: {end_drfold - start_drfold:.2f} sec")

            # === protenixで2構造生成（4,5番目として使う）===
            N_sample = 2
        else:
            # === protenixで5構造生成（全て）===
            N_sample = 5

        # === protenixによる予測 ===
        start_protenix = timer()
        configs["sample_diffusion"]["N_sample"] = N_sample
        test_df = pd.DataFrame([row])
        dataset = DictDataset(test_df.sequence, dump_dir='output', id_list=test_df.target_id, use_msa=False)

        try:
            data, atom_array, err = dataset[0]
            new_configs = update_inference_configs(configs, data["N_token"].item())
            runner.update_model_configs(new_configs)
            prediction = runner.predict(data)
            prediction = prediction['coordinate'][:, data['input_feature_dict']['atom_to_tokatom_idx'] == 12]
            prediction_np = prediction.cpu().numpy()

            for k in range(N_sample):
                xyz = prediction_np[k]  # (L, 3)
                all_coordinates.append(xyz)

            torch.cuda.empty_cache()
            end_protenix = timer()
            print(f"[{target_id}] Protenix prediction time: {end_protenix - start_protenix:.2f} sec")

        except Exception as e:
            print(f"Error with target_id {target_id} in protenix: {e}")
            for _ in range(N_sample):
                all_coordinates.append(np.zeros((L, 3)))

        # === 結果の整形と保存 ===
        start_output = timer()
        result = []
        for j, (resname, resid) in enumerate(zip(sequence, range(1, len(sequence) + 1))):
            row_data = [f"{target_id}_{resid}", resname, resid]
            for k in range(5):
                if k < len(all_coordinates):
                    x, y, z = all_coordinates[k][j]
                    row_data += [x, y, z]
                else:
                    row_data += [0.0, 0.0, 0.0]  # 補完
            result.append(row_data)

        columns = ['ID', 'resname', 'resid'] + [f"{a}_{i}" for i in range(1, 6) for a in ['x', 'y', 'z']]
        df = pd.DataFrame(result, columns=columns)
        df.to_csv('submission.csv', index=False, mode='a', header=(i == 0))
        end_output = timer()
        print(f"[{target_id}] Output save time: {end_output - start_output:.2f} sec")

    end_total = timer()
    print(f"Total prediction time: {end_total - start_total:.2f} sec")

In [42]:
run_prediction(valid_df1)

will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
train scheduler 16.0
inference scheduler 16.0
Diffusion Module has 16.0


Predicting:   0%|          | 0/12 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[R1107] DRfold2 prediction time: 32.38 sec


/kaggle/input/protenix/Protenix-main/protenix/openfold_local/model/primitives.py:238: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/kaggle/input/protenix/Protenix-main/protenix/openfold_local/model/primitives.py:215: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/kaggle/input/protenix/Protenix-main/protenix/openfold_local/model/primitives.py:280: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Predicting:   8%|▊         | 1/12 [01:10<12:52, 70.19s/it]

[R1107] Protenix prediction time: 37.80 sec
[R1107] Output save time: 0.01 sec
[R1108] DRfold2 prediction time: 30.34 sec


...

[R1156] Protenix prediction time: 48.31 sec
[R1156] Output save time: 0.01 sec
[R1189] DRfold2 prediction time: 62.75 sec


Predicting:  92%|█████████▏| 11/12 [1:28:11<06:48, 408.03s/it]

[R1189] Protenix prediction time: 39.64 sec
[R1189] Output save time: 0.00 sec
[R1190] DRfold2 prediction time: 62.78 sec


Predicting: 100%|██████████| 12/12 [1:29:53<00:00, 449.46s/it]

[R1190] Protenix prediction time: 39.62 sec
[R1190] Output save time: 0.00 sec
Total prediction time: 5421.56 sec


In [43]:
import pandas as pd
import numpy as np

# 読み込み
submission = pd.read_csv("submission.csv")

# target_id を取得（residを除いた部分すべてを対象にする）
submission['target_id'] = submission['ID'].apply(lambda x: '_'.join(x.split('_')[:-1]))
submission['resid'] = submission['resid'].astype(int)

# 準備（valid_df: 完全版, valid_df1: トリミング版）
valid_df = valid_df.copy()
valid_df1 = valid_df1.copy()

# 出力結果を格納
final_submission_rows = []

# カラム名の定義
coord_columns = [f"{a}_{i}" for i in range(1, 6) for a in ['x', 'y', 'z']]
columns = ['ID', 'resname', 'resid'] + coord_columns

# 各targetについて処理
for idx, row in valid_df.iterrows():
    target_id = row['target_id']
    sequence = row['sequence']
    full_length = len(sequence)

    # submission.csvから該当する構造を取得
    sub_df = submission[submission['target_id'] == target_id]

    # ID順（resid順）に並べる
    sub_df = sub_df.sort_values(by='resid')

    # 予測済みのresidのセット
    predicted_resids = set(sub_df['resid'].tolist())

    # 1からfull_lengthまで順に並べて座標を復元 or パディング
    for i in range(1, full_length + 1):
        resname = sequence[i - 1]
        match = sub_df[sub_df['resid'] == i]

        if not match.empty:
            coords = match[coord_columns].values[0].tolist()
        else:
            coords = [0.0] * (5 * 3)  # x1,y1,z1,...,x5,y5,z5

        row_data = [f"{target_id}_{i}", resname, i] + coords
        final_submission_rows.append(row_data)

# データフレームとして保存
submission_full = pd.DataFrame(final_submission_rows, columns=columns)
submission_full.to_csv("submission.csv", index=False)
print("✅ 正しく並べ直して submission.csv を保存しました。")

✅ 正しく並べ直して submission.csv を保存しました。
